# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


 Load the environment variables using dotenv. +(1 pt)

In [8]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [19]:
import dask.dataframe as dd



+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [27]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

if os.path.exists(PRICE_DATA):
   print("PRICE_DATA =", PRICE_DATA)
else:
    print(f"Directory does not exist: {PRICE_DATA}")
    
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
# Write your code below.

parquet_files

PRICE_DATA = ../../05_src/data/prices/


['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2013/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2013/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1980/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1980/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1987/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1987/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1989/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1989/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1988/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1988/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1986/part.0.parquet

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [37]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = dd_px.groupby("ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x["Close"].shift(1),
        Adj_Close_lag_1 = x["Adj Close"].shift(1),
        returns = (x["Close"] / x["Close"].shift(1)) - 1,
        hi_lo_range = x["High"] - x["Low"]
    )
)

dd_feat

/var/folders/dk/j3ll34kx4233h20p0d_ld2t80000gn/T/ipykernel_15220/2174717257.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby("ticker", group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=60,,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [41]:
# Write your code below.
dp_feat=dd_feat.compute()
dp_feat ['returns_ma_10'] = dp_feat['returns'].rolling(10).mean()



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Not necessarily. We can calculate a moving average directly in Dask, which is often better if the dataset is large.
+ Would it have been better to do it in Dask? Why?

Yes — in most cases, it would have been better to do it in Dask, especially for large datasets.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.